In [1]:
import numpy as np
import pandas as pd

In [167]:
PATH='data/transactions'
col_names = [
    'parkingFacilityId',
    'parkingFacilityZoneId',
    'parkingTransactionId',
    'pmsTicketReference',
    'pmsTicketId',
    'parkingTransactionTypeId',
    '_dateTimeEntry',
    'dateTimeEntry',
    '_dateTimeExit',
    'dateTimeExit',
    'duration',
    'stationEntryId',
    'stationExitId',
    'articleId',
    'productCategory',
    'productCategorySub1',
    'productCategorySub2',
    'licensePlate',
    'country',
    'pms'
]  

use_cols = [
    'parkingFacilityId',
    'parkingTransactionTypeId',
    'dateTimeEntry',
    'dateTimeExit',
    'duration'
] 

date_cols = [
    'dateTimeEntry',
    'dateTimeExit'
]

col_types = {
    'parkingFacilityId': str
}

def ms2dt(ms): return pd.to_datetime(ms, unit='ms')

transactions = pd.read_csv(f'{PATH}/transactions.csv', 
                           delimiter=';', 
                           header=None, 
                           names=col_names, 
                           usecols=use_cols,
                           parse_dates=date_cols,
                           date_parser=ms2dt,
                           dtype=col_types,
                           low_memory=False,
                           # nrows=100000
                          )
transactions['dateEntry'] = transactions['dateTimeEntry'].dt.date
transactions['dateExit'] = transactions['dateTimeExit'].dt.date
# transactions['duration'] = pd.to_timedelta(transactions['duration'], unit='ms')

In [168]:
complete_transactions = transactions[transactions['parkingTransactionTypeId'] == 'C']
complete_transactions = complete_transactions.drop(['parkingTransactionTypeId', 'dateTimeEntry', 'dateTimeExit', 'dateExit'], axis=1)

In [169]:
grouped = complete_transactions.groupby(['parkingFacilityId', 'dateEntry'])

In [173]:
txagg = grouped.agg([len, np.mean]).rename(columns={'len': 'count'})
txagg.reset_index(inplace=True)

In [192]:
txagg.columns = pd.Index(['facility', 'entry_date', 'count', 'avg_duration'])

In [193]:
txagg.to_feather(f'{PATH}/txagg')

In [194]:
df = pd.read_feather(f'{PATH}/txagg')

/home/koendej/apps/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [195]:
len(df)

101798

In [243]:
null_date = pd.to_datetime('1994-01-01').date()
entries = transactions[transactions['dateEntry'] != null_date]


In [244]:
entries = entries[(entries['parkingTransactionTypeId'] != 'C') | (entries['duration'] > 0)]

len(entries)

47765538

In [245]:
grouped_entries = entries.drop(['parkingTransactionTypeId', 'dateTimeEntry', 'dateTimeExit', 'dateExit'], axis=1)
grouped_entries = grouped_entries.groupby(['parkingFacilityId', 'dateEntry'])


In [246]:
entries_agg = grouped_entries.agg([len]).rename(columns={'len': 'count'})

In [247]:
entries_agg.reset_index(inplace=True)

In [248]:
entries_agg.columns = pd.Index(['facility', 'entry_date', 'count'])
entries_agg.to_feather(f'{PATH}/entries_agg')

In [249]:
entries_agg

,facility,entry_date,count
0,0500140,2015-12-16,2
1,0500140,2015-12-18,2
2,0500140,2015-12-20,1
3,0500140,2015-12-21,933
4,0500140,2015-12-22,1427
5,0500140,2015-12-23,1641
6,0500140,2015-12-24,2
7,0500140,2015-12-25,2
8,0500140,2015-12-26,20
9,0500140,2015-12-27,593
